In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


#length와 달리 랜덤으로 예제를 선택함   
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self,input_variables):
        from random import choice
        return [choice(self.examples)]

chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,  
    callbacks = [
        StreamingStdOutCallbackHandler(),
    ],
  
)


examples = [
            {"question": "France?", 
            "answer": """
                Here is what I know:,
                Capital: Paris,
                Language: French,
                Food: Wine and Cheese,
                Currency: Euro,
                """,
            },
            {
            "question": "Italy?",
            "answer": """
                I know this: ,
                Capital: Rome,
                Language: Italian,
                Food: Pizza and Pasta,
                Currency: Euro,
                """,
            },
            { 
            "question": "Greece?",
                "answer": """
                I know this: ,
                Capital: Athens,
                Language: Greek,
                Food: Souvlaki and Feta Cheese,
                Currency: Euro,
                """,
            },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# example_selector = LengthBasedExampleSelector( 
#     examples = examples,
#     example_prompt = example_prompt,
#     #예제의 갯수를 제한해서 돈을 절약할 수 있음
#     max_length = 500,
# )



example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables = ["country"],
)
prompt.format(country="Brazil")

#chain = prompt | chat
#chain.invoke({"country": "Turkey"})

'Human: France?\nAI:\n                Here is what I know:,\n                Capital: Paris,\n                Language: French,\n                Food: Wine and Cheese,\n                Currency: Euro,\n                \n\nHuman: What do you know about Brazil?'